# Regresión Semana 2: Regresión múltiple (descenso por gradiente)

En el primer cuaderno exploramos la regresión múltiple usando scikit-learn. Ahora usaremos la librería **pandas** junto con **numpy** para resolver los pesos de regresión con descenso de gradiente.

En este notebook cubriremos la estimación de los pesos de regresión múltiple mediante el descenso por gradiente.  harás:

* Añadir una columna constante de 1's para tener en cuenta el intercepto.
* Convertir un df en un array Numpy
* Escribir una función predict_output() usando Numpy
* Escribir una función numpy para calcular la derivada de los pesos de regresión con respecto a una única característica.
* Escribir una función de descenso del gradiente para calcular los pesos de regresión dado un vector inicial de pesos, tamaño de paso y tolerancia.
* Utilizar la función de descenso de gradiente para estimar los pesos de regresión para múltiples características.

In [1]:
#Importamos las librerías necesarias
import pandas as pd
import numpy as np

## Cargamos los datos

In [2]:
#Cargamos los datos

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 
              'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

df_train = pd.read_csv('/content/kc_house_train_data.csv', dtype= dtype_dict)

df_test = pd.read_csv('/content/kc_house_test_data.csv', dtype= dtype_dict)

df = pd.read_csv('/content/kc_house_data.csv', dtype= dtype_dict)

## Convertir a Numpy Array

Aunque **Pandas** ofrece una serie de beneficios a los usuarios (especialmente cuando se utiliza en Big Data) para entender los detalles de la implementación de algoritmos es importante trabajar con una librería que permita realizar operaciones matriciales de forma directa (y optimizada). Numpy es una solución de Python para trabajar con matrices (o cualquier "array" multidimensional).

El valor predicho dados los pesos y las características, es simplemente el **[producto escalar](https://es.wikipedia.org/wiki/Producto_escalar)** entre el vector de características y el de pesos. Del mismo modo, si colocamos todas las características fila por fila en una matriz, el valor predicho para todas las observaciones puede calcularse multiplicando la "matriz de características" por el "vector de pesos".

Primero tenemos que tomar el Dataframe de nuestros datos y convertirlo en un array 2D de numpy (también llamado **matriz**).

Ahora escribiremos una función que aceptará un DataFrame, una lista de nombres de columnas (por ejemplo ['sqft_living', 'bedrooms']) y una variable objetivo, por ejemplo ('price') y devolverá dos cosas:

* Una matriz numpy cuyas columnas son las características deseadas más una columna constante (así creamos un 'intercepto')
* Una matriz numpy que contiene los valores de la salida

In [3]:
def get_numpy_data(df, features, output):

    
    features_matrix = df[features] 
    
    # así es como se añade una columna constante a un df
    features_matrix['constant'] = 1
    
    # añade la columna 'constant' al principio de la lista de características para que podamos extraerla junto con las demás:
    #features.insert(0, 'constant')
    features = ['constant'] + features # Así es como combinamos dos listas

    # seleccionar las columnas de df dadas por la lista features en el df (ahora incluyendo la constante):
    features_matrix = features_matrix[features]

    #Convertimos el dataframe a un matriz de numpy
    features_matrix = np.array(features_matrix)

    #Cogemos las características para la salida
    output_array = df[output]

    #Convertimos el array a numpy
    output_array = np.array(output_array)
    
    return(features_matrix, output_array)


Para probar la función anterior, utilizamos la característica 'sqft_living' , una constante como nuestras características y el precio como nuestro output.

In [4]:
import warnings
warnings.filterwarnings('ignore')

(example_features, example_output) = get_numpy_data(df, ['sqft_living'], 'price') 
print(example_features[0,:])
print(example_output[0])


[1.00e+00 1.18e+03]
221900.0


## 2. Predicción del output dados los pesos de regresión



Supongamos que tenemos los pesos [1.0, 1.0] y las características [1.0, 1180.0] y queremos calcular la predicción de salida 1.0\*1.0 + 1.0\*1180.0 = 1181.0 esto es el producto escalar (**DOT PRODUCT**) entre estas dos matrices. Si son matrices numpy podemos utilizar np.dot() para calcular esto:

In [5]:
my_weights = np.array([1., 1.]) # pesos ejemplo
my_features = example_features[0,] # Utilizaremos el primer data point
predicted_value = np.dot(my_features, my_weights)
print(predicted_value)

1181.0


Si la matriz de características (incluyendo una columna de 1s para la constante) se almacena como un array 2D(o matriz) y los pesos de regresión se almacenan como un array 1D, entonces la salida predicha es simplemente el producto escalar entre la matriz de características y los pesos (con los pesos a la derecha). 

Escribe una función '**predict_output**' que acepte una matriz 2D 'feature_matrix' y una matriz 1D 'weights' y devuelva una matriz 1D 'predictions'.

In [6]:
def predict_output(feature_matrix, weights):
    #supongamos que feature_matrix es una matriz numpy que contiene las características como columnas y weights es la matriz numpy 
    # create el vector de predicciones utilizando np.dot()

    predictions = np.dot(feature_matrix, weights)
    return(predictions)


Si quieres testear tu código, ejecuta la siguiente celda

In [7]:
test_predictions = predict_output(example_features, my_weights)
print (test_predictions[0]) # debería ser 1181.0
print (test_predictions[1]) # debería 2571.0

1181.0
2571.0


# Cálculo de la Derivada

Ahora vamos a pasar a calcular la derivada de la función de coste de regresión. Recordemos que la función de coste es la Suma del Cuadrado de los resíduos entre el valor predicho y el valor real

Dado que la derivada de una suma es la suma de las derivadas, podemos calcular la derivada para un único punto de datos y, a continuación, sumar los puntos de datos. Podemos escribir la diferencia al cuadrado entre la salida observada y la salida predicha para un solo punto de la siguiente manera:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Donde tenemos k características y una constante. Así que la derivada con respecto al peso w[i] por la **[regla de la cadena](https://es.wikipedia.org/wiki/Regla_de_la_cadena)** es:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

El término dentro de la paréntesis es sólo el error (diferencia entre la predicción y el output). Así que podemos reescribir esto como:

2\*error\*[feature_i]

Es decir, la derivada del peso de la característica i es la suma (sobre los puntos datos) de 2 veces el producto del error y la propia característica. En el caso de la constante, es el doble de la suma de los errores.

Recordemos que el doble de la suma del producto de dos vectores es el doble del producto escalar de los dos vectores. Por lo tanto, la derivada para el peso de la característica_i es sólo dos veces el producto escalar entre los valores de la característica_i y los errores actuales. 

Con esto en mente complete la siguiente función derivada que calcula la derivada del peso dado el valor de la característica (sobre todos los puntos de datos) y los errores (sobre todos los puntos de datos).


In [8]:
def feature_derivative(errors, feature):
    # Supongamos que errors y feature son ambas matrices numpy de la misma longitud (número de puntos de datos).
    # calcula el doble del producto punto de estos vectores como 'derivada' y devuelve el valor

    derivative = 2*np.dot(errors, feature)
    return(derivative)

Para probar su feature_derivative ejecuta lo siguiente:

In [9]:
(example_features, example_output) = get_numpy_data(df, ['sqft_living'], 'price') 

my_weights = np.array([0., 0.]) # Esto hace todas las predicciones 0

test_predictions = predict_output(example_features, my_weights) 

# 2 arrays numpy se pueden restar  con '-':
errors = test_predictions - example_output #Errores de predicción en este caso es solo el example_output

feature = example_features[:,0] #calculemos la derivada con respecto a 'constant', el ":" indica "todas las filas"

derivative = feature_derivative(errors, feature)
print (derivative)

print (-np.sum(example_output)*2) # debería dar lo mismo que arriba

-23345850016.0
-23345850016.0


## Descenso del gradiente

Ahora escribiremos una función que realice un descenso de gradiente. La premisa básica es simple. Dado un punto de partida actualizamos los pesos actuales moviéndonos en la dirección del gradiente negativo. Recordemos que el gradiente es la dirección de *incremento* y por lo tanto el gradiente negativo es la dirección de *decrecimiento* y estamos tratando de *minimizar* una función de coste. 

La cantidad en la que nos movemos en la *dirección* del gradiente negativo se denomina "stepsize" o "eta". Nos detenemos cuando estamos "suficientemente cerca" del óptimo. Definimos esto requiriendo que la magnitud (longitud) del vector gradiente sea menor que un " umbral de tolerancia" fijo.

Teniendo esto en cuenta, completa la siguiente función de descenso del gradiente utilizando tu función derivada anterior. Para cada paso en el descenso de gradiente actualizamos el peso de cada característica antes de calcular nuestro criterio de parada

In [10]:
from math import sqrt # recuerda que la magnitud/longitud de un vector [g[0], g[1], g[2]] es sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [11]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights)  #asegúrate que es un np array
    while not converged:

        #calcule las predicciones basadas en la feature_matrix y los pesos utilizando su función predict_output()
        #COMPLETAR
        predictions = predict_output(feature_matrix, weights)
        
        # computar los errores como predictions - output
        errors = predictions - output

        gradient_sum_squares = 0 #inicializar el gradiente sum_of_squares

        for i in range(len(weights)): #iteramos sobre los pesos


            # Recordemos que feature_matrix[:, i] es la columna de características asociada a weights[i]
            # calcula la derivada para weights[i]:
            #COMPLETAR

            derivative = feature_derivative(errors, feature_matrix[:, i])  
            
            # suma el valor cuadrado de la derivada a la suma de cuadrados del gradiente (para evaluar la convergencia)
            #COMPLETAR
            gradient_sum_squares += (derivative**2)

            # restar el stepsize multiplicado por la derivada del peso actual
            #COMPLETAR
            weights[i] -= (step_size * derivative)
      
            # calcular la raíz cuadrada de la suma de cuadrados (gradient_sum_squares) del gradiente para obtener la magnitud del gradiente:
            gradient_magnitude = sqrt(gradient_sum_squares)

            
            
            
        
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

Hay que tener en cuenta algunas cosas antes de ejecutar el descenso de gradiente. Dado que el gradiente es una suma de todos los puntos de datos e implica un producto de un error y una característica, el gradiente en sí será muy grande, ya que las características son grandes (squarefeets) y la salida es grande (price). Por lo tanto, aunque se puede esperar que la "tolerancia" sea pequeña, lo es sólo en relación con el tamaño de las características. 

Por razones similares, el stepsize será mucho menor de lo que cabría esperar, pero esto se debe a que el gradiente tiene valores muy grandes.

# Ejecutando el Descenso Gradiente como Regresión Simple

Aunque el descenso del gradiente está diseñado para la regresión múltiple, ya que la constante es ahora una característica, podemos utilizar la función de descenso del gradiente para estimar los parámetros en la regresión simple en **sqft**. La siguiente celda establece la feature_matriz, el output, los pesos iniciales y el 'stepsize' para el primer modelo:

In [12]:
# Vamos a testear el descenso del gradiente
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(df_train, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

A continuación, ejecute el descenso del gradiente con los parámetros anteriores.

In [13]:
#COMPLETAR
test_weight = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)
print(test_weight)


[-46999.88716555    281.91211918]


Utiliza los pesos recién estimados y la función predict_output() para calcular las predicciones sobre todos los datos de TEST (primero tendrás que crear un numpy array con el test de feature_matrix y test_output primero

In [14]:
simple_features = ['sqft_living']
my_output = 'price'
(test_simple_feature_matrix, test_output) = get_numpy_data(df_test, simple_features, my_output)


In [15]:
test_predictions = predict_output(test_simple_feature_matrix, test_weight)
print("Example:",test_predictions[0])


Example: 356134.4432550024


Ahora que tiene las predicciones sobre los datos de test, calcule el RSS sobre el conjunto de test. Guarde este valor para compararlo más adelante. Recuerde que el RSS es la suma de los errores al cuadrado (diferencia entre la predicción y el output).



In [ ]:
test_residuals = test_output - test_predictions

test_RSS = (test_residuals * test_residuals).sum()
print(test_RSS)


275400044902128.3


# Ejecutar una regresión múltiple

Ahora utilizaremos más de una característica real. Utilice el siguiente 
código para producir los pesos  (weights) de un segundo modelo con los siguientes parámetros:

In [16]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 es la media de pies cuadrados de los 15 vecinos más cercanos.
my_output = 'price'
(feature_matrix, multi_output) = get_numpy_data(df_train, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9


Utilice los parámetros anteriores para estimar los pesos del modelo. GUARDA LOS VALORES EN UNA VARIABLE.

In [ ]:
multi_weights = regression_gradient_descent(feature_matrix, multi_output, initial_weights, step_size, tolerance)
print(multi_weights)

[-9.99999688e+04  2.45072603e+02  6.52795267e+01]


In [ ]:
multi_weights

array([-9.99999688e+04,  2.45072603e+02,  6.52795267e+01])

Usa tus pesos recién estimados y la función predict_output para calcular las predicciones en los datos de TEST. No olvides crear primero una matriz numpy para estas características del conjunto de prueba.

In [ ]:
(test_multi_feature_matrix, multi_output) = get_numpy_data(df_test, model_features, my_output)
multi_predictions = predict_output(test_multi_feature_matrix, multi_weights)

print(multi_predictions)

[366651.41162949 762662.39850726 386312.09557541 ... 682087.39916306
 585579.27901327 216559.20391786]


**Pregunta: ¿Cuál es el precio previsto para la primera vivienda del conjunto de datos de PRUEBA para el modelo 2 (redondee al dólar más próximo)?**

In [ ]:
print (multi_predictions[0])


366651.4116294939


¿Cuál es el precio real de la primera vivienda del conjunto de datos de prueba?

In [ ]:
df_test['price'][0]


310000.0

Pregunta: ¿Qué estimación se acercó más al precio real de la 1ª casa en el conjunto de datos de TEST, el modelo 1 o el modelo 2?

Utilice ahora sus predicciones y el OUTPUT para calcular el RSS del modelo 2 en los datos de PRUEBA.

In [ ]:
print ('la predicción del primer modelo es 356134 $ y la del segundo modelo es 3666651 $')

la predicción del primer modelo es 356134 $ y la del segundo modelo es 3666651 $


Pregunta: ¿Qué modelo (1 ó 2) tiene el RSS más bajo en todos los datos de TEST?

In [ ]:
print('RSS del primer modelo es 2,75400047593e+14 y RSS del segundo modelo es 2,70263446465e+14".')

RSS del primer modelo es 2,75400047593e+14 y RSS del segundo modelo es 2,70263446465e+14".
